https://www.youtube.com/watch?v=f4s1h2YETNY

In [2]:
# Install headless OpenGL and virtual display
!apt-get install -y xvfb libosmesa6-dev libgl1-mesa-glx
!pip install pyopengl pyvirtualdisplay imageio[ffmpeg] moderngl glfw Pillow

from pyvirtualdisplay import Display
import moderngl
import numpy as np
from PIL import Image
import imageio
from IPython.display import Video

# Start virtual display (needed for OpenGL context in Colab)
display = Display(visible=0, size=(800, 600))
display.start()

# Now create ModernGL context
ctx = moderngl.create_standalone_context()

# Vertex shader
vertex_shader = """
#version 330
in vec2 in_vert;
out vec2 v_text;
void main() {
    v_text = in_vert;
    gl_Position = vec4(in_vert, 0.0, 1.0);
}
"""

# Fragment shader (your code)
fragment_shader = """
#version 330
out vec4 fragColor;
in vec2 v_text;
uniform vec2 iResolution;
uniform float iTime;

vec3 palette(float t) {
    return 0.5 + 0.5*cos(6.2831*(vec3(0.3,0.3,0.3)*t + vec3(0.0,0.1,0.2)));
}

void main() {
    vec2 uv = v_text * iResolution / iResolution.y;
    vec2 uv0 = uv;
    vec3 finalColor = vec3(0.0);

    for (float i = 0.0; i < 3.0; i++) {
        uv = fract(uv * 1.5) - 0.5;
        float d = length(uv) * exp(-length(uv0));
        vec3 col = palette(length(uv0) + iTime * .4);
        d = sin(d*8. + iTime) / 8.;
        d = abs(d);
        d = 0.01 / d;
        finalColor += col * d;
    }

    fragColor = vec4(finalColor, 1.0);
}
"""

prog = ctx.program(vertex_shader=vertex_shader, fragment_shader=fragment_shader)

# Fullscreen quad
vertices = np.array([
    -1.0, -1.0,
     1.0, -1.0,
    -1.0,  1.0,
     1.0,  1.0,
], dtype='f4')
vbo = ctx.buffer(vertices)
vao = ctx.simple_vertex_array(prog, vbo, 'in_vert')

# Render function
def render_frame(time=0.0, res=(512, 512)):
    fbo = ctx.simple_framebuffer(res)
    fbo.use()
    ctx.clear(0.0, 0.0, 0.0, 1.0)
    prog['iResolution'].value = res
    prog['iTime'].value = time
    vao.render(moderngl.TRIANGLE_STRIP)
    data = fbo.read(components=3)
    img = Image.frombytes('RGB', res, data).transpose(Image.FLIP_TOP_BOTTOM)
    return img

# Generate frames
frames = []
fps = 30
duration = 5
for i in range(duration * fps):
    t = i / fps
    img = render_frame(time=t)
    frames.append(np.array(img))

# Save video
video_path = "shader.mp4"
imageio.mimsave(video_path, frames, fps=fps)
Video(video_path, embed=True)


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
libgl1-mesa-glx is already the newest version (23.0.4-0ubuntu1~22.04.1).
xvfb is already the newest version (2:21.1.4-2ubuntu1.7~22.04.15).
The following additional packages will be installed:
  libdrm-dev libgl-dev libglx-dev libosmesa6 libpciaccess-dev mesa-common-dev
The following NEW packages will be installed:
  libdrm-dev libgl-dev libglx-dev libosmesa6 libosmesa6-dev libpciaccess-dev
  mesa-common-dev
0 upgraded, 7 newly installed, 0 to remove and 35 not upgraded.
Need to get 5,761 kB of archives.
After this operation, 18.5 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/main amd64 libpciaccess-dev amd64 0.16-3 [21.9 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 libdrm-dev amd64 2.4.113-2~ubuntu0.22.04.1 [292 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy/main amd64 libglx-dev amd64 1.4.0-1 [14.1 kB]
Get:4 http://archive.

In [3]:
# Install headless OpenGL + helpers
!apt-get install -y xvfb libosmesa6-dev libgl1-mesa-glx
!pip install pyopengl pyvirtualdisplay imageio[ffmpeg] moderngl glfw Pillow

from pyvirtualdisplay import Display
import moderngl
import numpy as np
from PIL import Image
import imageio
from IPython.display import Video

# Start virtual display
display = Display(visible=0, size=(800, 600))
display.start()

# Create ModernGL context
ctx = moderngl.create_standalone_context()

# Vertex shader
vertex_shader = """
#version 330
in vec2 in_vert;
out vec2 v_text;
void main() {
    v_text = in_vert;
    gl_Position = vec4(in_vert, 0.0, 1.0);
}
"""

# Fragment shader (updated version from your screenshot)
fragment_shader = """
#version 330
out vec4 fragColor;
in vec2 v_text;
uniform vec2 iResolution;
uniform float iTime;

vec3 palette(float t) {
    return 0.5 + 0.5*cos(6.2831*(vec3(0.3,0.3,0.3)*t + vec3(0.0,0.1,0.2)));
}

void main() {
    vec2 uv = v_text * iResolution / iResolution.y;
    vec2 uv0 = uv;
    vec3 finalColor = vec3(0.0);

    for (float i = 0.0; i < 4.0; i++) {
        uv = fract(uv * 1.5) - 0.5;

        float d = length(uv) * exp(-length(uv0));

        vec3 col = palette(length(uv0) + i*0.4 + iTime*0.4);

        d = sin(d*8. + iTime) / 8.;
        d = abs(d);

        d = 0.01 / d;

        finalColor += col * d;
    }

    fragColor = vec4(finalColor, 1.0);
}
"""

# Compile shaders
prog = ctx.program(vertex_shader=vertex_shader, fragment_shader=fragment_shader)

# Fullscreen quad
vertices = np.array([
    -1.0, -1.0,
     1.0, -1.0,
    -1.0,  1.0,
     1.0,  1.0,
], dtype='f4')
vbo = ctx.buffer(vertices)
vao = ctx.simple_vertex_array(prog, vbo, 'in_vert')

# Render function
def render_frame(time=0.0, res=(512, 512)):
    fbo = ctx.simple_framebuffer(res)
    fbo.use()
    ctx.clear(0.0, 0.0, 0.0, 1.0)
    prog['iResolution'].value = res
    prog['iTime'].value = time
    vao.render(moderngl.TRIANGLE_STRIP)
    data = fbo.read(components=3)
    img = Image.frombytes('RGB', res, data).transpose(Image.FLIP_TOP_BOTTOM)
    return img

# Generate frames
frames = []
fps = 30
duration = 6  # seconds
for i in range(duration * fps):
    t = i / fps
    img = render_frame(time=t)
    frames.append(np.array(img))

# Save video
video_path = "shader_kaleidoscope.mp4"
imageio.mimsave(video_path, frames, fps=fps)
Video(video_path, embed=True)


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
libosmesa6-dev is already the newest version (23.2.1-1ubuntu3.1~22.04.3).
libgl1-mesa-glx is already the newest version (23.0.4-0ubuntu1~22.04.1).
xvfb is already the newest version (2:21.1.4-2ubuntu1.7~22.04.15).
0 upgraded, 0 newly installed, 0 to remove and 35 not upgraded.
